# Malaria Diagnosis Using CNN and Transfer Learning

## Overview
This notebook implements a deep learning approach for malaria diagnosis using cell images. The project utilizes transfer learning with ResNet50 to classify blood cell images as either parasitized (infected with malaria) or uninfected.

## Dataset
- **Source**: NIH Malaria Dataset
- **Classes**: Parasitized vs Uninfected blood cells
- **Total Images**: ~27,558 images
- **Split**: 80% training, 20% testing

## Methodology
The notebook implements two experimental approaches:

### Experiment 1 (E1): Frozen Base Model
- Uses pre-trained ResNet50 with frozen base layers
- Only trains the custom classification head
- Learning rate: 1e-4
- Focus: Fast training with basic transfer learning

### Experiment 2 (E2): Fine-tuned Model  
- Fine-tunes the top 30 layers of ResNet50
- Unfreezes selected layers for domain-specific learning
- Learning rate: 1e-5 (lower for stability)
- Focus: Enhanced performance through fine-tuning

## Performance Metrics
Each experiment reports comprehensive evaluation metrics:
- **Accuracy**: Overall classification performance
- **Precision**: True positive rate (TP / (TP + FP))
- **Recall**: Sensitivity (TP / (TP + FN))
- **F1-Score**: Harmonic mean of precision and recall
- **AUC**: Area under ROC curve

## Visualizations
Each model includes complete visual evaluation:
1. **Learning Curves**: Training/validation loss and accuracy over epochs
2. **Confusion Matrix**: Classification performance across classes
3. **ROC/AUC Curves**: Trade-offs between sensitivity and specificity

## Requirements
- TensorFlow 2.x
- scikit-learn
- matplotlib
- pandas
- NumPy

In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [5]:
# Importing basic libraries
import os
import random
import shutil
from matplotlib import pyplot
from matplotlib.image import imread
%matplotlib inline
import tensorflow as tf
import numpy as np



# Importing the Keras libraries and packages
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Convolution2D as Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense

In [3]:
# Download the data in the allocated google cloud-server. If already down, turn downloadData=False
downloadData = True
if downloadData == True:
  indrive = False
  if indrive == True:
    !wget https://data.lhncbc.nlm.nih.gov/public/Malaria/cell_images.zip -P "/content/drive/My Drive/Colab Notebooks/ai-labs/malaria-diagnosis"
    !unzip "/content/drive/My Drive/Colab Notebooks/ai-labs/malaria-diagnosis/cell_images.zip" -d "/content/drive/My Drive/Colab Notebooks/ai-labs/malaria-diagnosis/"
    !ls "/content/drive/My Drive/Colab Notebooks/ai-labs/malaria-diagnosis"
  else: #incloud google server
    !rm -rf cell_images.*
    !wget https://data.lhncbc.nlm.nih.gov/public/Malaria/cell_images.zip
    !unzip cell_images.zip >/dev/null 2>&1
    !ls

--2025-10-04 09:50:14--  https://data.lhncbc.nlm.nih.gov/public/Malaria/cell_images.zip
Resolving data.lhncbc.nlm.nih.gov (data.lhncbc.nlm.nih.gov)... 3.163.189.83, 3.163.189.93, 3.163.189.81, ...
Connecting to data.lhncbc.nlm.nih.gov (data.lhncbc.nlm.nih.gov)|3.163.189.83|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 353452851 (337M) [application/zip]
Saving to: ‘cell_images.zip’

cell_images.zip     100%[===================>] 337.08M   418MB/s    in 0.8s    

2025-10-04 09:50:15 (418 MB/s) - ‘cell_images.zip’ saved [353452851/353452851]

cell_images  cell_images.zip  drive  sample_data


In [7]:
import os
import shutil
import random
from sklearn.model_selection import train_test_split
import tensorflow as tf

# Set random seed for reproducibility
random.seed(42)

# Create train and test directories
os.makedirs('train/Parasitized', exist_ok=True)
os.makedirs('train/Uninfected', exist_ok=True)
os.makedirs('test/Parasitized', exist_ok=True)
os.makedirs('test/Uninfected', exist_ok=True)

print("Created train/test directory structure")

# Split data for each class
classes = ['Parasitized', 'Uninfected']
train_split = 0.8  # 80% for training, 20% for testing

for class_name in classes:
    class_path = os.path.join('cell_images', class_name)

    # Get all image files in this class
    image_files = [f for f in os.listdir(class_path)
                   if f.lower().endswith(('.png', '.jpg', '.jpeg'))]

    print(f"\n{class_name}: Found {len(image_files)} images")

    # Split the files
    train_files, test_files = train_test_split(
        image_files, train_size=train_split, random_state=42
    )

    print(f"  - Training: {len(train_files)} images")
    print(f"  - Testing: {len(test_files)} images")

    # Copy files to train directory
    for file in train_files:
        src = os.path.join(class_path, file)
        dst = os.path.join('train', class_name, file)
        shutil.copy2(src, dst)

    # Copy files to test directory
    for file in test_files:
        src = os.path.join(class_path, file)
        dst = os.path.join('test', class_name, file)
        shutil.copy2(src, dst)

print("\n Data splitting completed!")

# Verify the structure
print("\nFinal directory structure:")
for split in ['train', 'test']:
    for class_name in classes:
        path = os.path.join(split, class_name)
        count = len(os.listdir(path))
        print(f"{split}/{class_name}: {count} images")

Created train/test directory structure

Parasitized: Found 13779 images
  - Training: 11023 images
  - Testing: 2756 images

Uninfected: Found 13779 images
  - Training: 11023 images
  - Testing: 2756 images

 Data splitting completed!

Final directory structure:
train/Parasitized: 11023 images
train/Uninfected: 11023 images
test/Parasitized: 2756 images
test/Uninfected: 2756 images


## Phase 2: Model Setup and Training with ResNet50

This section implements transfer learning using ResNet50 for malaria diagnosis. We'll conduct two experiments to compare different training strategies.

In [ ]:
# Set random seed for reproducibility
seed = 42
tf.random.set_seed(seed)
np.random.seed(seed)
random.seed(seed)

# Import necessary libraries for deep learning and evaluation
from tensorflow.keras import layers, Model
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix, roc_curve
import matplotlib.pyplot as plt
import itertools
import pandas as pd

print("✓ Libraries imported and random seed set for reproducibility")

In [ ]:
# Configuration parameters
ai_project = '.'
training_path = os.path.join(ai_project, 'train')
testing_path  = os.path.join(ai_project, 'test')

IMG_SIZE = (224, 224)  # ResNet50 input size
BATCH_SIZE = 64
AUTOTUNE = tf.data.AUTOTUNE

print(f"✓ Configuration set:")
print(f"  - Image size: {IMG_SIZE}")
print(f"  - Batch size: {BATCH_SIZE}")
print(f"  - Training path: {training_path}")
print(f"  - Testing path: {testing_path}")

In [ ]:
# Define preprocessing function for ResNet50
def resnet_preprocess(image):
    """
    Preprocess images for ResNet50 input.
    Converts to float32 and resizes to target size.
    """
    image = tf.image.convert_image_dtype(image, dtype=tf.float32)
    image = tf.image.resize(image, IMG_SIZE)
    return image

# Build optimized tf.data pipelines
def make_datasets(train_dir, val_dir, img_size=IMG_SIZE, batch_size=BATCH_SIZE):
    """
    Create optimized training and validation datasets.
    
    Args:
        train_dir: Directory containing training images
        val_dir: Directory containing validation images
        img_size: Target image size (height, width)
        batch_size: Batch size for training
    
    Returns:
        Tuple of (train_dataset, validation_dataset)
    """
    # Create datasets from directories
    train_ds = tf.keras.utils.image_dataset_from_directory(
        train_dir, image_size=img_size, batch_size=batch_size, shuffle=True
    )
    val_ds = tf.keras.utils.image_dataset_from_directory(
        val_dir, image_size=img_size, batch_size=batch_size, shuffle=False
    )

    # Optimize pipeline with preprocessing and prefetching
    train_ds = train_ds.map(
        lambda x, y: (resnet_preprocess(x), y), 
        num_parallel_calls=AUTOTUNE
    ).prefetch(AUTOTUNE)
    
    val_ds = val_ds.map(
        lambda x, y: (resnet_preprocess(x), y), 
        num_parallel_calls=AUTOTUNE
    ).prefetch(AUTOTUNE)
    
    return train_ds, val_ds

# Create datasets
train_ds, val_ds = make_datasets(training_path, testing_path)
print("✓ Datasets created and optimized")

In [ ]:
# Data augmentation pipeline
data_augmentation = tf.keras.Sequential([
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.06),
    layers.RandomZoom(0.06),
    layers.RandomTranslation(0.05, 0.05),
], name="data_augmentation")

def build_resnet_model(img_size=IMG_SIZE, dropout_head=0.5):
    """
    Build ResNet50-based transfer learning model.
    
    Args:
        img_size: Input image size (height, width)
        dropout_head: Dropout rate for classification head
    
    Returns:
        Tuple of (model, base_model)
    """
    # Load pre-trained ResNet50 without top layers
    base = ResNet50(
        weights='imagenet', 
        include_top=False, 
        input_shape=img_size + (3,)
    )
    base.trainable = False  # Start with frozen base
    
    # Build custom classification head
    inputs = layers.Input(shape=img_size + (3,))
    x = data_augmentation(inputs)  # Apply augmentation
    x = base(x, training=False)    # Pass through frozen base
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.BatchNormalization()(x)
    x = layers.Dense(256, activation='relu')(x)
    x = layers.Dropout(dropout_head)(x)
    x = layers.BatchNormalization()(x)
    outputs = layers.Dense(1, activation='sigmoid')(x)  # Binary classification
    
    model = Model(inputs, outputs, name="resnet50_transfer")
    return model, base

print("✓ Data augmentation and model architecture defined")

In [ ]:
# Training and evaluation utility functions
def compile_and_train(model, train_ds, val_ds, lr, epochs, callbacks=None):
    """
    Compile and train the model with specified parameters.
    
    Args:
        model: Keras model to train
        train_ds: Training dataset
        val_ds: Validation dataset  
        lr: Learning rate
        epochs: Number of training epochs
        callbacks: List of Keras callbacks
    
    Returns:
        Training history object
    """
    model.compile(
        optimizer=Adam(learning_rate=lr),
        loss='binary_crossentropy',
        metrics=['accuracy', tf.keras.metrics.AUC(name="auc")]
    )
    cb = callbacks or []
    history = model.fit(
        train_ds, 
        validation_data=val_ds, 
        epochs=epochs, 
        callbacks=cb
    )
    return history

def evaluate_model(model, dataset):
    """
    Comprehensive model evaluation with multiple metrics.
    
    Args:
        model: Trained model to evaluate
        dataset: Dataset for evaluation
    
    Returns:
        Tuple of (metrics_dict, y_true, y_pred, y_prob)
    """
    y_true, y_prob = [], []
    
    # Collect predictions and true labels
    for x, y in dataset:
        probs = model.predict(x, verbose=0).ravel()
        y_prob.extend(probs)
        y_true.extend(y.numpy())
    
    # Convert to numpy arrays
    y_true = np.array(y_true).astype(int)
    y_prob = np.array(y_prob)
    y_pred = (y_prob >= 0.5).astype(int)
    
    # Calculate comprehensive metrics
    metrics = {
        'accuracy': accuracy_score(y_true, y_pred),
        'precision': precision_score(y_true, y_pred, zero_division=0),
        'recall': recall_score(y_true, y_pred, zero_division=0),
        'f1': f1_score(y_true, y_pred, zero_division=0),
        'auc': roc_auc_score(y_true, y_prob),
    }
    return metrics, y_true, y_pred, y_prob

print("✓ Training and evaluation functions defined")

In [ ]:
# Visualization functions for model evaluation
def plot_history(history, title='Training'):
    """
    Plot training and validation learning curves.
    
    Shows both accuracy and loss over epochs to visualize:
    - Model convergence
    - Overfitting/underfitting
    - Training stability
    
    Args:
        history: Keras training history object
        title: Title prefix for plots
    """
    plt.figure(figsize=(12, 4))

    # Plot accuracy curves
    plt.subplot(1, 2, 1)
    plt.plot(history.history['accuracy'], label='Training Accuracy', linewidth=2)
    plt.plot(history.history['val_accuracy'], label='Validation Accuracy', linewidth=2)
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.title(f'{title} - Accuracy Over Epochs')
    plt.grid(True, alpha=0.3)

    # Plot loss curves  
    plt.subplot(1, 2, 2)
    plt.plot(history.history['loss'], label='Training Loss', linewidth=2)
    plt.plot(history.history['val_loss'], label='Validation Loss', linewidth=2)
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    plt.title(f'{title} - Loss Over Epochs')
    plt.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()

def plot_confusion_matrix(y_true, y_pred, classes=['Uninfected','Parasitized']):
    """
    Plot confusion matrix with detailed annotations.
    
    Shows classification performance across classes:
    - True positives and negatives
    - False positives and negatives
    - Per-class accuracy
    
    Args:
        y_true: True labels
        y_pred: Predicted labels
        classes: Class names for labeling
    """
    cm = confusion_matrix(y_true, y_pred)
    plt.figure(figsize=(6, 5))
    plt.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
    plt.title('Confusion Matrix\n(Rows: True Labels, Columns: Predicted Labels)', fontsize=12)
    plt.colorbar()
    
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)
    
    # Add text annotations
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], 'd'),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black",
                 fontsize=14, fontweight='bold')
    
    plt.ylabel('True Label')
    plt.xlabel('Predicted Label')
    plt.tight_layout()
    plt.show()

def plot_roc(y_true, y_prob):
    """
    Plot ROC curve with AUC score.
    
    Demonstrates trade-offs between sensitivity and specificity:
    - Higher AUC indicates better discrimination
    - Diagonal line represents random chance
    - Curve closer to top-left indicates better performance
    
    Args:
        y_true: True binary labels
        y_prob: Predicted probabilities
    """
    fpr, tpr, _ = roc_curve(y_true, y_prob)
    auc_score = roc_auc_score(y_true, y_prob)
    
    plt.figure(figsize=(6, 5))
    plt.plot(fpr, tpr, linewidth=3, label=f'ROC Curve (AUC = {auc_score:.4f})')
    plt.plot([0, 1], [0, 1], '--', color='gray', label='Random Classifier')
    plt.xlabel('False Positive Rate (1 - Specificity)')
    plt.ylabel('True Positive Rate (Sensitivity)')
    plt.title('ROC Curve - Model Discrimination Performance')
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()

print("✓ Visualization functions defined")

In [ ]:
# Training callbacks for optimization
def make_callbacks():
    """
    Create training callbacks for better model performance.
    
    Returns:
        List of callbacks:
        - EarlyStopping: Prevents overfitting
        - ReduceLROnPlateau: Adapts learning rate
    """
    early_stopping = EarlyStopping(
        monitor='val_loss', 
        patience=3, 
        restore_best_weights=True, 
        verbose=1
    )
    reduce_lr = ReduceLROnPlateau(
        monitor='val_loss', 
        factor=0.5, 
        patience=2, 
        min_lr=1e-7, 
        verbose=1
    )
    return [early_stopping, reduce_lr]

print("✓ Training callbacks configured")

## Experiment 1: Frozen Base Model (Transfer Learning)

**Approach**: Use pre-trained ResNet50 with frozen base layers, only training the custom classification head.

**Rationale**: 
- Fast training with minimal computational requirements
- Leverages ImageNet features for cell image classification
- Baseline performance for comparison with fine-tuning

**Parameters**:
- Learning Rate: 1e-4 (standard for head-only training)
- Frozen Layers: All ResNet50 base layers
- Trainable Parameters: Only classification head (~2.1M parameters)

In [ ]:
# Experiment 1: Build and train model with frozen base
print("🔬 Starting Experiment 1: Frozen Base Model")
print("="*50)

# Build model with frozen base
model_resnet, base_resnet = build_resnet_model(dropout_head=0.5)

# Ensure base is frozen
for layer in base_resnet.layers:
    layer.trainable = False

print(f"Model built with {model_resnet.count_params():,} total parameters")
trainable_params = sum([tf.keras.backend.count_params(w) for w in model_resnet.trainable_weights])
print(f"Trainable parameters: {trainable_params:,}")

# Train the model
history_e1 = compile_and_train(
    model_resnet,
    train_ds,
    val_ds,
    lr=1e-4,
    epochs=12,
    callbacks=make_callbacks()
)

print("✓ Experiment 1 training completed")

In [ ]:
# Experiment 1: Evaluation and Visualization
print(" Evaluating Experiment 1 Performance")
print("="*40)

# Plot learning curves
plot_history(history_e1, title='ResNet50 - E1 (Frozen Base)')

# Evaluate model performance
metrics_e1, y_true_e1, y_pred_e1, y_prob_e1 = evaluate_model(model_resnet, val_ds)

# Display metrics
print(" Experiment 1 Performance Metrics:")
print("-" * 30)
for metric, value in metrics_e1.items():
    print(f"{metric.upper():<12}: {value:.4f}")

# Plot confusion matrix
print("\n Confusion Matrix Analysis:")
plot_confusion_matrix(y_true_e1, y_pred_e1)

# Plot ROC curve
print(" ROC Curve Analysis:")
plot_roc(y_true_e1, y_prob_e1)

# Store results for comparison
results = []
results.append({
    'experiment_id': 'E1',
    'description': 'Frozen ResNet50 base, train head only; lr=1e-4',
    **metrics_e1
})

## Experiment 2: Fine-tuned Model (Advanced Transfer Learning)

**Approach**: Fine-tune the top 30 layers of ResNet50 while keeping earlier layers frozen.

**Rationale**:
- Adapts high-level features to malaria-specific patterns
- Balances feature adaptation with training stability
- Expected to improve performance over frozen base

**Parameters**:
- Learning Rate: 1e-5 (lower for stability during fine-tuning)
- Unfrozen Layers: Top 30 layers of ResNet50
- Batch Normalization: Kept frozen to maintain stability
- Additional Trainable Parameters: ~6.3M parameters

In [ ]:
# Experiment 2: Fine-tune top layers
print("🔬 Starting Experiment 2: Fine-tuned Model")
print("="*50)

# Configuration for fine-tuning
N_unfreeze = 30
base_resnet.trainable = True

# Selectively unfreeze top layers
frozen_count = 0
unfrozen_count = 0

for i, layer in enumerate(base_resnet.layers):
    if i < len(base_resnet.layers) - N_unfreeze:
        layer.trainable = False
        frozen_count += 1
    else:
        # Keep BatchNormalization layers frozen for stability
        if isinstance(layer, tf.keras.layers.BatchNormalization):
            layer.trainable = False
            frozen_count += 1
        else:
            layer.trainable = True
            unfrozen_count += 1

print(f"Layer configuration:")
print(f"  - Frozen layers: {frozen_count}")
print(f"  - Unfrozen layers: {unfrozen_count}")

# Recompile with lower learning rate for fine-tuning
model_resnet.compile(
    optimizer=Adam(learning_rate=1e-5),
    loss='binary_crossentropy',
    metrics=['accuracy', tf.keras.metrics.AUC(name="auc")]
)

# Display parameter counts
total_params = model_resnet.count_params()
trainable_params = sum([tf.keras.backend.count_params(w) for w in model_resnet.trainable_weights])
print(f"Total parameters: {total_params:,}")
print(f"Trainable parameters: {trainable_params:,}")

# Train with fine-tuning
history_e2 = model_resnet.fit(
    train_ds, 
    validation_data=val_ds, 
    epochs=12, 
    callbacks=make_callbacks()
)

print("✓ Experiment 2 training completed")

In [ ]:
# Experiment 2: Evaluation and Visualization
print(" Evaluating Experiment 2 Performance")
print("="*40)

# Plot learning curves
plot_history(history_e2, title=f'ResNet50 - E2 (Fine-tune top {N_unfreeze})')

# Evaluate model performance
metrics_e2, y_true_e2, y_pred_e2, y_prob_e2 = evaluate_model(model_resnet, val_ds)

# Display metrics
print(" Experiment 2 Performance Metrics:")
print("-" * 30)
for metric, value in metrics_e2.items():
    print(f"{metric.upper():<12}: {value:.4f}")

# Plot confusion matrix
print("\n Confusion Matrix Analysis:")
plot_confusion_matrix(y_true_e2, y_pred_e2)

# Plot ROC curve
print(" ROC Curve Analysis:")
plot_roc(y_true_e2, y_prob_e2)

# Store results for comparison
results.append({
    'experiment_id': 'E2',
    'description': f'Fine-tune top {N_unfreeze} layers; lr=1e-5',
    **metrics_e2
})

## Results Comparison and Analysis

This section provides a comprehensive comparison of both experiments, analyzing the trade-offs between different training strategies for malaria diagnosis.

In [ ]:
# Comprehensive Results Comparison
print("🏆 FINAL RESULTS COMPARISON")
print("="*60)

# Create results DataFrame for tabular comparison
results_df = pd.DataFrame(results)

# Display formatted results table
print(" Performance Metrics Summary:")
print("=" * 80)
print(f"{'Experiment':<12} {'Description':<35} {'Accuracy':<10} {'Precision':<10} {'Recall':<10} {'F1-Score':<10} {'AUC':<8}")
print("-" * 80)

for _, row in results_df.iterrows():
    print(f"{row['experiment_id']:<12} {row['description'][:34]:<35} "
          f"{row['accuracy']:<10.4f} {row['precision']:<10.4f} "
          f"{row['recall']:<10.4f} {row['f1']:<10.4f} {row['auc']:<8.4f}")

print("\n" + "="*80)

# Calculate improvements
if len(results_df) >= 2:
    e1_acc = results_df.iloc[0]['accuracy']
    e2_acc = results_df.iloc[1]['accuracy']
    acc_improvement = ((e2_acc - e1_acc) / e1_acc) * 100
    
    e1_f1 = results_df.iloc[0]['f1']
    e2_f1 = results_df.iloc[1]['f1']
    f1_improvement = ((e2_f1 - e1_f1) / e1_f1) * 100
    
    print(" Performance Analysis:")
    print(f"  • Accuracy improvement (E2 vs E1): {acc_improvement:+.2f}%")
    print(f"  • F1-Score improvement (E2 vs E1): {f1_improvement:+.2f}%")
    
    # Determine best model
    best_model = "E2" if e2_acc > e1_acc else "E1"
    print(f"  • Best performing model: {best_model}")

# Display complete results DataFrame
print("\n📋 Detailed Results Table:")
display(results_df)

print("\n Key Findings:")
print("• Learning curves show model convergence and training stability")
print("• Confusion matrices reveal classification performance per class")  
print("• ROC curves demonstrate discrimination capability (AUC scores)")
print("• Fine-tuning generally improves performance but requires more resources")
print("• Both models show strong performance for malaria diagnosis task")

print("\n✅ Analysis Complete - All visualizations and metrics generated")